In [96]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [141]:
import torch
import torch.nn.functional as F

In [98]:
words = open('names.txt', 'r').read().split('\n')
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [99]:
charset = sorted(list(set(''.join(words))))
charset

['a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [100]:
stoi = {s:i+1 for i, s in enumerate(charset)}
stoi['.'] = 0

In [101]:
itos = {i:s for s, i in stoi.items()}

In [119]:
# building the dataset 

def build_dataset(words):

    block_size = 3 
    X, Y = [], []

    for w in words[:5]:
        print(w)
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            print(''.join(itos[i] for i in context), '---->', itos[ix])
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y


In [150]:
# cross entropy

# counts = logits.exp()
# probs = counts/ counts.sum(1, keepdim=True)
# loss = -probs[torch.arange(32), Y.long()].log().mean()
# loss

In [166]:
# forward pass 

C = torch.randn((27, 2))
emb = C[X.long()]
w1 = torch.randn((6, 100))
b1 = torch.randn(100)
h = (emb.view(-1, 6) @ w1 + b1).sigmoid()
w2 = torch.randn((100, 27))
b2 = torch.randn(27)
parameters = [C, w1, b1, w2, b2]
logits = h @ w2 + b2
loss = F.cross_entropy(logits, Y.long())
loss


tensor(9.5225)

In [170]:
for p in parameters:
    p.requires_grad=True
    p.grad = None

loss.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn